In [14]:
# from pydub import AudioSegment
# import os

# for folder in os.listdir('small_dataset/'):
#     folder_path = os.path.join('small_dataset/', folder)
#     for file in os.listdir(folder_path):
#         file_path = os.path.join(folder_path,file)
#         sound = AudioSegment.from_mp3(file_path)
#         sound.export(file_path.removesuffix(".mp3")+".wav", format="wav")
#         os.remove(file_path)

In [29]:
# import librosa

# audio, _ = librosa.load('/Users/fouk/Documents/GR_Folk_Music_Classification/rnn/small_dataset/Epirus/vasilarchontissa.wav', sr=None)

In [1]:
import pandas as pd
from ast import literal_eval
literals = {
  'mfcc':  literal_eval,
  'chroma': literal_eval
}
df = pd.read_csv('rnn_features.csv', converters=literals)

In [2]:
import numpy as np
df = df[['label', 'mfcc', 'chroma']]

In [3]:
df.groupby('label').size()

label
Romania            6
Southern-Italy    29
Thessaly          20
Thrace            31
Turkey             4
dtype: int64

In [3]:
df['mfcc'] = df['mfcc'].apply(lambda x: np.array(x))
df['chroma'] = df['chroma'].apply(lambda x: np.array(x))

In [4]:
dataset = df.values
labels = dataset[:, 0]
dataset = dataset[:, 1:]

In [5]:
# Create an empty list to store the concatenated arrays
concatenated_data = []

# Iterate over each pair of arrays in the data
for array1, array2 in dataset:
    # Concatenate the arrays along the first dimension (axis 0)
    concatenated_array = np.concatenate((array1, array2), axis=0)
    # Append the concatenated array to the list
    concatenated_data.append(concatenated_array.T)

# Convert the list of concatenated arrays to a numpy array
final_data = np.array(concatenated_data)

In [6]:
dataset = final_data

In [7]:
dataset.shape

(90, 100, 25)

In [8]:
encoding = {
    'Romania': 0,          
    'Southern-Italy': 1,  
    'Thessaly': 2,         
    'Thrace': 3,           
    'Turkey': 4
}

In [9]:
labels2 = [0] * len(labels)
for i,label in enumerate(labels):
    labels2[i] = encoding[label]

In [10]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from sound_utils import *
from data_utils import *
from model import *
from model_utils import ModelUtils

num_classes = 5 
dataset = AudioDataset(dataset, labels2)

# Define the sizes for the splits
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)

input_shape = 25

In [11]:
model_experiment = ModelUtils(train_loader,val_loader,test_loader,input_shape,num_classes)

epochs = 1
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

params = {
            'learning_rate': 0.0001,
            'num_layers': 2,
            'skip_connections': False,
            'dropout_probability': 0.5,
            'hidden_layers': 2,
            'gradient_clipping': False,
            'cell_type': 'rnn',
            'hidden_size': 128,
            'bidirectional': False
            }

model_experiment.train_evaluate(params)

Epoch 2/1: 100%|██████████| 13/13 [00:04<00:00,  2.87batch/s, loss=1.6] 


End of Epoch: 1  |  Train Loss: 1.567  |  Val Loss: 1.520  |  Train Acc: 34.62%  |  Val Acc: 48.89%


RNNModel(
  (rnn_layer): RNN(25, 128, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (dense1): Linear(in_features=128, out_features=128, bias=True)
  (dense2): Linear(in_features=128, out_features=5, bias=True)
  (relu): ReLU()
)

In [12]:
model_experiment.test_model()

--------------------------------------------
Model confusion matrix: 
 [[0 0 0 1]
 [0 0 0 8]
 [0 0 0 2]
 [0 0 0 4]]
--------------------------------------------
Model classification report: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00         2
           3       0.27      1.00      0.42         4

    accuracy                           0.27        15
   macro avg       0.07      0.25      0.11        15
weighted avg       0.07      0.27      0.11        15

--------------------------------------------
Model accuracy:  0.26666666666666666
--------------------------------------------


/Applications/anaconda3/envs/ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

ValueError: multiclass format is not supported

In [1]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from sound_utils import *
from data_utils import *
from model import *
from model_utils import ModelUtils

##############################################################
##############################################################

# SEED = 13 # reproducible results: Same results in every run
# IN_PATH = ''
# DATA_PATH = '' 
# OUT_PATH = ''
# EPOCH = 20 # number of epochs to run for model

# np.random.seed(SEED) 
# torch.manual_seed(SEED)
# torch.cuda.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True  # cuda algorithms
# os.environ['PYTHONHASHSEED'] = str(SEED)

#plt.style.use('seaborn') # use seaborn style plotting

##############################################################
##############################################################


# Example usage:
# wav_folder = './train/'
# frame_length = 44100  # Example: 1 second at 44.1 kHz sampling rate
# max_lentgh = 9500000
num_classes = 5  # Example: 3 classes for classification

# # Load data for training
# input_data, labels = DataUtils.load_data_for_training(wav_folder, frame_length, max_lentgh)

# # Preprocess the data
# #input_data, labels = DataUtils.preprocess_data(input_data, labels, frame_length)

# # Create a dataset and data loader
dataset = AudioDataset(dataset, labels)

# Define the sizes for the splits
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)

input_shape = 25
# Build the RNN model
#RNNModel(input_shape=(frame_length, 1), num_classes=num_classes).to(device)

#TODO: FIX DATALOADERS
model_experiment = ModelUtils(train_loader,val_loader,test_loader,input_shape,num_classes)


device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# Define loss function and optimizer

# model_experiment.model = RNNModel(input_shape, num_classes, 0, 
#                                 False, 1, None, 
#                                 None).to(device)
# model = model_experiment.model
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 1

# model_experiment.criterion = criterion
# model_experiment.optimizer = optimizer

# model_experiment.train_epoch(epochs)

params = {
            'learning_rate': 0.0001,
            'num_layers': 2,
            'skip_connections': False,
            'dropout_probability': 0.5,
            'hidden_layers': 2,
            'gradient_clipping': False,
            'cell_type': 'rnn',
            'hidden_size': 128,
            'bidirectional': False
            }

model_experiment.train_evaluate(params)

# Optionally, save the trained model
# torch.save(model.state_dict(), 'path_to_save_model.pth')


NameError: name 'dataset' is not defined

In [2]:
model_experiment.test_model()

--------------------------------------------
Model confusion matrix: 
 [[1 2 3]
 [0 0 0]
 [0 0 0]]
--------------------------------------------
Model classification report: 
               precision    recall  f1-score   support

           0       1.00      0.17      0.29         6
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0

    accuracy                           0.17         6
   macro avg       0.33      0.06      0.10         6
weighted avg       1.00      0.17      0.29         6

--------------------------------------------
Model accuracy:  0.16666666666666666
--------------------------------------------
--------------------------------------------


/Applications/anaconda3/envs/ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applicati

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [2]:
model_experiment.model

RNNModel(
  (rnn_layer): LSTM(44100, 128, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (dense): Linear(in_features=256, out_features=3, bias=True)
  (relu): ReLU()
)

In [2]:
input_data[3][-1]

array([1., 1., 1., ..., 0., 0., 0.], dtype=float32)

In [11]:
from sound_utils import *
audio = SoundUtils.load_wav('/Users/fouk/Documents/GR_Folk_Music_Classification/rnn/train/siganos-kontylies.wav')

In [10]:
audio.shape

(9702000,)

In [12]:
audio.shape

(8334900,)